<a href="https://www.kaggle.com/code/rishabhsingh18/model-ark-experiment?scriptVersionId=188201066" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
from torchvisions.datasets import ImageFolder
from torch.utils.data import DataLoader

# data preparation
def load_preprocess_data(data_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = ImageFolder(root=data_path, transform-transform)
    data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    return data_loader
# insert path to the datasets in ''
data_loader = load_preprocess_data('/kaggle/input/nih-chest-x-ray-14-224x224-resized/Data_Entry_2017.csv')

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class StudentTeacherModel(nn.Module):
    def __init__(self, base_model):
        super(StudentTeacherModel, self).__init__()
        self.base_model = base_model
        self.classification_head = nn.Linear(base_model.fc.in_features, 14)
        self.segmentation_head = nn.Conv2d(base_model.fc.in_feature, 1, kernel_size=1)
        
    def forward(self, x):
        features = self.base_model(x)
        classification_output = self.classification_head(features)
        segmentation_output = self.segmentation_head(features.unsqueeze(2).unsqueeze(3))
        return classfication_output, segmentation_output

base_model = models.resnet50(pretrained=True)
model = StudentTeacherModel(base_model)

In [ ]:
import torch.optim as optim

criterion - nn